In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

## load dataset

In [12]:
members=pd.read_csv("members_FY.csv")

In [42]:
tender=pd.read_csv("tender_type.csv",sep = "|", encoding = "ISO-8859-1")

In [2]:
pos=pd.read_csv("pos.csv",sep = "|", encoding = "ISO-8859-1")

## get number of visit

In [3]:
num_visit=pos[['MEMBERSHIP_ID','VISIT_NBR']].groupby('VISIT_NBR').count()
num_visit=pd.merge(pos[['MEMBERSHIP_ID','VISIT_NBR']],num_visit,how='left',on='VISIT_NBR')
num_visit=num_visit.rename(index=str,columns={"MEMBERSHIP_ID_x":"MEMBERSHIP_ID","VISIT_NBR":"VISIT_NBR","MEMBERSHIP_ID_y":"num_visit"})

## Get a new table with visit information: frequency, time

In [5]:
#people have same visit number but different visit time
pos_visit=pos[['VISIT_DATE','VISIT_NBR']]
#pos_visit=pd.merge(pos_visit,num_visit,how='left',on='VISIT_NBR')
pos_visit=pos_visit.drop_duplicates(subset=['VISIT_NBR'])

In [7]:
pos_visit=pd.merge(pos_visit,num_visit,how='left', on='VISIT_NBR')

In [21]:
pos_visit=pos_visit.drop_duplicates(subset=['MEMBERSHIP_ID'])
members_simple=members[['MEMBERSHIP_ID','RENEW_IND','PLUS_STATUS_BEFORE_REN']]
visit=pd.merge(members_simple,pos_visit,how='left',on='MEMBERSHIP_ID')

In [41]:
#visit.to_csv("visit.csv")

## clean "members" dataset

In [185]:
#Clean members dataset again
members=members.drop(columns=['Unnamed: 0','MEMBERSHIP_TYPE_DESC','COHORT_MONTH','LAST_RENEW_DATE','RENEW_DATE','NEXT_RENEW_DATE','JOIN_DATE',"PLUS_UPGRADE_DATE"])

In [186]:
# encode the label of categorical data
#income: low:0,middle:1,high:2
# Westrn European & Eastrn European & Greek & Scandinavian:0,African Americn & Native Americn & Hispanic：1,
#Korean & Chinese& Asian Other & Vietnamese& Jewish&Middle Eastern:2,Polynesian:3
replace_col={"marital_status_desc":{"Unknown":"NaN","Single":0,"Married":1},
            "PLUS_STATUS_BEFORE_REN":{"BASE":1,"PLUS":2},
            "PLUS_STATUS_AFTER_REN":{"BASE":1,"PLUS":2},
            "PLUS_MEMBERSHIP_IND":{"N":0,"Y":1},
            "autorenew_ind":{"N":0,"Y":1},
            "payroll_deduct_ind":{"N":0,"Y":1},
            "RENEW_IND":{2:1},
            "income_desc":{"$50K - $74K":1,'$150K +':2,'< $15K':0,'$35K - $49K':1,'$75K - $99K':1,'$25K - $34K':0,
                          '$100K-$124K':2,'$15K - $24K':0,'$125K-$149K':2,'Unknown':"NaN"},
            "ethnic_desc":{'Unknown':"NaN",'Westrn European':0,'Eastrn European':0,'Greek':0,'Scandinavian':0,
                          'African Americn':1,'Native Americn':1,'Hispanic':1,'Korean':2,'Chinese':2,
                          'Jewish':2,'Vietnamese':2,'Asian Other':2,'Middle Eastern':2,'Polynesian':3}}
members.replace(replace_col,inplace=True)
#deal with the hhh_age_desc
#members.hhh_age_desc=members.hhh_age_desc.str.extract('(\d+)')
#I use R to categorize the age and use python to load the dataset back
#MEMBERSHIP_TYPE_CODE: 2 for Savings and 3 for Business
members=pd.read_csv("mb1.csv")
members=members.drop(columns=['Unnamed: 0','X',"MEMBERSHIP_TYPE_DESC"])
members.head(10)

,MEMBERSHIP_ID,MEMBERSHIP_TYPE_CODE,MEMBERSHIP_TYPE_DESC,MILES_TO_CLUB,hhh_age_desc,marital_status_desc,income_desc,hh_size_desc,nbr_children_desc,ethnic_desc,TENURE_GRP,RENEW_IND,PLUS_STATUS_BEFORE_REN,PLUS_STATUS_AFTER_REN,PLUS_MEMBERSHIP_IND,autorenew_ind,payroll_deduct_ind
0,55081,2,SAVINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,2,NaN,0,0,0
1,168584,2,SAVINGS,2.80,Age 76 - 77,NaN,1,2.0,0.0,0,1,0,2,NaN,1,0,0
2,211504,3,BUSINESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,2,NaN,1,0,0
3,39986,2,SAVINGS,1.15,Age 54 - 55,0,2,1.0,0.0,0,1,1,2,2.0,1,0,0
4,260907,2,SAVINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,NaN,0,0,0
5,163426,2,SAVINGS,32.62,Age 30 - 31,1,0,3.0,0.0,0,1,0,1,NaN,0,0,0
6,79292,2,SAVINGS,20.92,Age 24 - 25,1,1,6.0,2.0,0,1,1,1,1.0,0,1,0
7,38280,2,SAVINGS,1.71,Age 98 - 99,NaN,1,1.0,0.0,0,1,1,1,1.0,0,0,0
8,4040,2,SAVINGS,22.07,Age 72 - 73,1,1,3.0,0.0,0,1,0,1,NaN,0,0,0
9,113302,3,BUSINESS,6.74,Age 58 - 59,1,0,2.0,0.0,0,1,1,2,2.0,0,0,0


## Create a summary of "pos" dataset

In [103]:
cat_name=pos[['MEMBERSHIP_ID','CATEGORY_DESC']]
#create a pivot table to summarize the purchase categories of each customer
table = pd.pivot_table(cat_name,index = ['MEMBERSHIP_ID'],columns=['CATEGORY_DESC'], aggfunc = lambda x : len(x))

pos['spending']=pos['RETAIL_PRICE']*pos["UNIT_QTY"]
pos_summary=pos.groupby('MEMBERSHIP_ID')['spending'].sum()
pos_summary=pd.DataFrame(pos_summary)
pos_summary['num_visit']=pos.groupby('MEMBERSHIP_ID')['VISIT_NBR'].count()
pos_summary['num_purchase_type']=pos.groupby('MEMBERSHIP_ID')['PRIMARY_DESC'].nunique()

data=pd.merge(pos_summary,table,how='left',on='MEMBERSHIP_ID')# merge the two dataset and create the final cleaned pos

In [234]:
comb_data=pd.merge(members,data,how='left',on='MEMBERSHIP_ID')# combine the members and summarized pos
comb_data=comb_data.drop(columns=['num_visit'])
comb_data=pd.merge(comb_data,pos_visit,how='left',on='MEMBERSHIP_ID')
#comb_data.to_csv("Final_data")
#comb_data=pd.read_csv("Final_data.csv")

In [25]:
comb_data=pd.read_csv("Final_data.csv")

## Get the payment method table

In [43]:
pos_visit_nbr=pos[['MEMBERSHIP_ID','VISIT_NBR']]
pos_payment=pd.merge(pos_visit_nbr,tender,how='left',on='VISIT_NBR')

# Categorize the payment method into three categories: cash, credit, saving
#cash:0,credit:1,saving:2
pay_cat={'TENDER_TYPE_DESC':{'Mastercard                              ':1,
                            'Debit Card                              ':2,
                            'Cash (change)                           ':0,
                            'Visa                                    ':1,
                            'Sams Consumer Credit                    ':1,
                            'Rx third party sales                    ':1,
                            'Shopping Card                           ':2,
                            'EBT Foodstamps                          ':2,
                            'Associate Discount Card                 ':1,
                            'American Express                        ':1,
                            'Discover                                ':1,
                            'Coupon                                  ':2,
                            'Electronic Check (ECA)                  ':2,
                            'Wal-Mart Credit Card                    ':1,
                            'Sams Business Credit                    ':1,
                            'Unknown                                 ':"NaN",
                            'DOTCOM                                  ':1,
                            'Check                                   ':2,
                            'Service Income                          ':2,
                            'Payroll Check                           ':2,
                            'EBT Cash                                ':0,
                            'Fleet                                   ':"NaN", 
                             'Coin Voucher                            ':2,
                             'Sams Direct Credit                      ':1,
                            'nan':'NaN'}}
pos_payment.replace(pay_cat,inplace=True)
pos_payment_method=pos_payment[['MEMBERSHIP_ID','TENDER_TYPE_DESC']]

In [46]:
renew_id=comb_data[['MEMBERSHIP_ID','RENEW_IND','PLUS_STATUS_BEFORE_REN']]
renew_pay=pd.merge(pos_payment_method,renew_id,how='left',on='MEMBERSHIP_ID')
renew_pay_type=renew_pay[['MEMBERSHIP_ID','TENDER_TYPE_DESC']]
renew_pay_type=renew_pay_type[renew_pay_type.TENDER_TYPE_DESC.isin([1,2,0])]# Choose only those who pay in those methods
renew_pay_type=renew_pay_type[['MEMBERSHIP_ID','TENDER_TYPE_DESC']]
#create a pivot table to transpose the payment method type for each member and count together
table_pay = pd.pivot_table(renew_pay_type,index = ['MEMBERSHIP_ID'],columns=['TENDER_TYPE_DESC'], aggfunc = lambda x : len(x))
rp=renew_pay[['MEMBERSHIP_ID','RENEW_IND','PLUS_STATUS_BEFORE_REN']]

In [49]:
table_pay.head(10)

TENDER_TYPE_DESC,0,1,2
MEMBERSHIP_ID,,,
1,207.0,142.0,226.0
2,67.0,222.0,193.0
3,1.0,141.0,387.0
4,65.0,159.0,327.0
6,27.0,58.0,220.0
7,NaN,29.0,112.0
8,9.0,NaN,54.0
9,37.0,43.0,49.0
10,10.0,172.0,37.0


In [19]:
pay_renew=pd.merge(rp,table_pay,how='left',on='MEMBERSHIP_ID')
pr=pay_renew.drop_duplicates(subset=['MEMBERSHIP_ID'])
pr=pr.rename(index=str,columns={0.0:'cash',1.0:"credit",2.0:"saving"})
pr.groupby('RENEW_IND')['cash','credit','saving'].sum()# the payment method sum of those who renew and not renew

# Seperate dataset into basic and plus

In [34]:
basic_data=comb_data[comb_data['PLUS_STATUS_BEFORE_REN']==1]
plus_data=comb_data[comb_data['PLUS_STATUS_BEFORE_REN']==2]

## Basic

In [35]:
list(basic_data)
basic_no_na=basic_data.dropna(subset = ['marital_status_desc','MILES_TO_CLUB','income_desc','ethnic_desc','nbr_children_desc','hh_size_desc'])
basic_no_na.reset_index(inplace = True)
basic_no_na.drop(['index'],axis = 1,inplace = True)
basic_no_na.fillna(0,inplace = True)
#basic_no_na.to_csv("basic_no_na.csv")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


## Plus

In [38]:
plus_no_na=plus_data.dropna(subset = ['marital_status_desc','MILES_TO_CLUB','income_desc','ethnic_desc','nbr_children_desc','hh_size_desc'])
plus_no_na.reset_index(inplace = True)
plus_no_na.drop(['index'],axis = 1,inplace = True)
plus_no_na.fillna(0,inplace = True)
#plus_no_na.to_csv("plus_no_na")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [19]:
#area = pd.merge(pos[['MEMBERSHIP_ID','MARKET_AREA_NAME']],members[['MEMBERSHIP_ID','RENEW_IND','PLUS_STATUS_BEFORE_REN']],how='left',on='MEMBERSHIP_ID')
#area.to_csv("area")

In [8]:
pos_simple=pos[['MEMBERSHIP_ID','VISIT_DATE','VISIT_TIME','CATEGORY_DESC']]

In [13]:
members_simple=members[['MEMBERSHIP_ID','RENEW_IND','PLUS_STATUS_BEFORE_REN']]

In [14]:
time=pd.merge(pos_simple,members_simple,how='left',on='MEMBERSHIP_ID')